In [ ]:
%matplotlib inline
import pathlib as pl
import numpy as np
import sys
import xugrid
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import rasterio.warp
from shapely.geometry import shape

import scipy.sparse as sparse

import flopy
import flopy.plot.styles as styles

from gdptools import WeightGenP2P


In [ ]:
sys.path.append("../common")
from liss_settings import cx, cx_provider, extent, boxx, boxy, get_dflow_grid_name, get_modflow_coupling_tag

In [ ]:
# Liv will update the coastal model to replace the coarse model
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

In [ ]:
get_modflow_coupling_tag(1.)

## Read the D-Flow FM output file

Make sure you run D-Flow FM by itself first so that there is an output NetCDF file available so that the mapping is done using the internal node order

In [ ]:
# use an output file because this is what will be available from bmi and is in the correct order
# NOTE: KLJ - we don't know where this FlowFM_map.nc comes from, but "run" doesn't exist yet...
source_path = "../dflow-fm/coarse/tides/FlowFM_map.nc"
source_ds = xugrid.open_dataset(source_path)

In [ ]:
source_ds

In [ ]:
print(source_ds.grid.face_node_connectivity.shape)
source_ds.grid.face_node_connectivity

(8323, 4)


array([[   1,    2,    0, -999],
       [   1,    0,    3, -999],
       [   4,    2,    1, -999],
       ...,
       [4845, 4844, 4846, -999],
       [4845, 4846, 4847, -999],
       [4847, 4846, 4916, -999]], dtype=int64)

In [ ]:
print(source_ds.grid.node_face_connectivity.shape)
source_ds.grid.node_face_connectivity

### Convert the NetCDF data to a geodataframe

In [ ]:
source_gdf = source_ds["mesh2d_nFaces"].ugrid.to_geodataframe(name="cell")
source_gdf.plot()

In [ ]:
source_gdf.set_crs(32618, inplace=True)

## Open the shapefile with the location of the coastal boundaries in MODFLOW

The shapefile needs to be limited to coastal boundary locations and be in the same coordinate system as the D-Flow FM model (UTM 18N).

In [ ]:
target_coastal = gpd.read_file("../modflow/gis/greenpoint_ghb_utm18n.shp") # this is the shapefile with coastal boundary conditions
target_coastal

In [ ]:
target_coastal.crs

In [ ]:
ax = target_coastal.plot(alpha=0.25, column="ghb_no")
cx.add_basemap(ax, crs=target_coastal.crs, attribution=False, source=cx_provider)

## Create the D-FLOW FM to GHB mapping

In [ ]:
# generate the weights
weight_gen = WeightGenP2P(
    target_poly=target_coastal,
    target_poly_idx="ghb_no",
    source_poly=source_gdf,
    source_poly_idx=["cell"],
    method="serial",
    weight_gen_crs=32618,
)
weights = weight_gen.calculate_weights()

In [ ]:
weights[:12]

In [ ]:
map_shape = (target_coastal.shape[0], source_gdf.shape[0]) # # of ghbs shapes, # of dflow fm shapes
map_shape

In [ ]:
dflow2mfghb = np.zeros(map_shape, dtype=float)
print(f"{dflow2mfghb.shape}\n{dflow2mfghb}")

In [ ]:
for r,c,v in zip(weights["ghb_no"], weights["cell"], weights["wght"]):
    print(r,c, v)
    dflow2mfghb[int(r),int(c)] = v

In [ ]:
cropped = dflow2mfghb[0:734, 1500:1600]

fig, ax = plt.subplots()
im = ax.imshow(cropped, origin='upper')
plt.colorbar(im, ax=ax)
ax.set_title("Cropped view of dflow2mfghb")
plt.show()


## Create the ghb masking array

Where the sums of the weights along a row are not equal to ~1.0

In [ ]:
# Mask where rows approximately sum to 1 
mask_idx = np.isclose(dflow2mfghb.sum(axis=1), 1.0)

print(f"{mask_idx.sum()}\n{mask_idx.shape}\n{mask_idx}")

### Test the D-FLOW FM to GHB mapping

In [ ]:
s = np.full(source_gdf.shape[0], 1.0)
h = np.full(mask_idx.shape, 2.0)
# Multiplies the matrix dflow2mfghb with the vector s. Since s is all 1.0s, this is equivalent to summing each row of dflow2mfghb.
# Then, it assigns these values only at positions where mask_idx is True in the array h.
h[mask_idx] = dflow2mfghb.dot(s)[mask_idx]
s.shape, dflow2mfghb.shape, h.shape

In [ ]:
print(f"{h.sum()}\n{h}")

#### Test with a nan

In [ ]:
s = np.random.random(source_gdf.shape[0])
s[1544] = -1e30
print(s)

In [ ]:
h = np.full(mask_idx.shape, 2.0)
h = dflow2mfghb.dot(s)
h.shape

In [ ]:
print(f"{h.sum()}\n{h}")

In [ ]:
dflow2mfghb

## Create the GHB to Qext mapping

In [ ]:
ghb2qext = np.transpose(dflow2mfghb.copy())

In [ ]:
ghb2qext

### Test the GHB to Qext mapping

In [ ]:
q = np.full(ghb2qext.shape[1], 1.0)

In [ ]:
qext = ghb2qext.dot(q)

In [ ]:
print(f"{qext.sum()}\n{qext.shape}")

## Save the mapping arrays

In [ ]:
fpath = f"../mapping/dflow2mfghb_{grid_name}.npz"
np.savez_compressed(fpath, dflow2mfghb=dflow2mfghb, ghbmask=mask_idx, ghb2qext=ghb2qext)